In [15]:
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout, Activation 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras import backend as K
from sklearn.metrics import classification_report,confusion_matrix
import cv2
import os
import numpy as np

In [16]:
labels = ['mamukka', 'lalettan']
img_width, img_height = 150, 150
train_data_dir = 'Data/train'
val_data_dir = 'Data/validation'
train = 1200
test = 150
epochs = 100
batch_size = 20
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3) #150, 150, 3


In [17]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

In [18]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [19]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 2635 images belonging to 2 classes.


In [20]:
val_generator = test_datagen.flow_from_directory(
    val_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 370 images belonging to 2 classes.


In [21]:
#creating neural network
cast = Sequential()
cast.add(Conv2D(32, (3,3), input_shape=input_shape))
cast.add(Activation('relu'))
cast.add(MaxPool2D(pool_size=(2,2)))

cast.summary()

cast.add(Conv2D(32, (3,3)))
cast.add(Activation('relu'))
cast.add(MaxPool2D(pool_size=(2,2)))

cast.add(Conv2D(64, (3,3)))
cast.add(Activation('relu'))
cast.add(MaxPool2D(pool_size=(2,2)))

cast.add(Flatten())
cast.add(Dense(64))         
cast.add(Activation('relu'))
cast.add(Dropout(0.5))
cast.add(Dense(1))
cast.add(Activation('sigmoid'))

cast.summary()

cast.compile(loss = 'binary_crossentropy',
            optimizer = 'rmsprop',
            metrics = ['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 148, 148, 32)      0         
__________________________________

In [22]:
#Augmentation configuration for training

cast.fit_generator(
    train_generator,
    steps_per_epoch = 2635 // batch_size,
    epochs = epochs,
    validation_data = val_generator,
    validation_steps = 370 // batch_size
)
cast.save('cast.h5')
